In [92]:
import os
import numpy as np
from pathlib import Path

import pandas as pd
import sys
# kouchou-aiのパスを追加
sys.path.append("../kouchou-ai/server")
sys.path.append("../kouchou-ai/server/broadlistening/pipeline/")

In [93]:
slug = "rep-election-2"
exp_prefix = f"20250415_ベースプロンプト_{slug}"
DATA_DIR = Path("../data/original_outputs")  # 広聴AIで出力したデータを保存しているディレクトリ
OUTPUT_DIR = Path("../data/experiment_outputs")

# クラスタリング

In [94]:
from umap import UMAP
from broadlistening.pipeline.steps.hierarchical_clustering import hierarchical_clustering_embeddings

arguments_df = pd.read_csv(DATA_DIR / slug /  "args.csv")
embeddings_df = pd.read_pickle(DATA_DIR / slug / "embeddings.pkl")
embeddings_array = np.asarray(embeddings_df["embedding"].values.tolist())
cluster_nums = [15, 70]
n_samples = embeddings_array.shape[0]
default_n_neighbors = 15

if n_samples <= default_n_neighbors:
    n_neighbors = max(2, n_samples - 1)
else:
    n_neighbors = default_n_neighbors

umap_model = UMAP(random_state=42, n_components=2, n_neighbors=n_neighbors)
umap_embeds = umap_model.fit_transform(embeddings_array)


/Users/nasuka/.pyenv/versions/3.12.4/lib/python3.12/site-packages/sklearn/utils/deprecation.py:151: FutureWarning: 'force_all_finite' was renamed to 'ensure_all_finite' in 1.6 and will be removed in 1.8.
  warnings.warn(
/Users/nasuka/.pyenv/versions/3.12.4/lib/python3.12/site-packages/umap/umap_.py:1952: UserWarning: n_jobs value 1 overridden to 1 by setting random_state. Use no seed for parallelism.
  warn(


In [95]:
# 広聴AIで出力したレポートのslug
cluster_results = hierarchical_clustering_embeddings(
    umap_embeds=umap_embeds,
    cluster_nums=cluster_nums,
)
result_df = pd.DataFrame(
    {
        "arg-id": arguments_df["arg-id"],
        "argument": arguments_df["argument"],
        "x": umap_embeds[:, 0],
        "y": umap_embeds[:, 1],
    }
)
for cluster_level, final_labels in enumerate(cluster_results.values(), start=1):
    result_df[f"cluster-level-{cluster_level}-id"] = [f"{cluster_level}_{label}" for label in final_labels]



start initial clustering
end initial clustering
start hierarchical clustering
[15, 70]
n_cluster_cut:  15
end hierarchical clustering


In [96]:
clusters_argument_df =  result_df.copy()
orig_initial_label_df = pd.read_csv(DATA_DIR / slug / "hierarchical_initial_labels.csv")
orig_merge_label_df = pd.read_csv(DATA_DIR / slug / "hierarchical_merge_labels.csv")

exp_output_dir = OUTPUT_DIR / exp_prefix

os.makedirs(exp_output_dir, exist_ok=True)


# プロンプト修正の実験

## 初期ラベリング

In [97]:
INITIAL_PROMPT = """あなたはKJ法が得意なデータ分析者です。userのinputはグループに集まったラベルです。なぜそのラベルが一つのグループであるか解説して、それから表札をつけてください。
出力はJSONとし、フォーマットは以下のサンプルを参考にしてください。


# サンプルの入出力
## 入力例
最近、政治家が能登の復興に向けた具体的なプランを発表し、地域の未来に明るい希望が見えてきました。市民として、真摯な取り組みに感謝しています。
災害復興支援が、選挙期間中にしっかり議論されるようになり、政治家が国民の本当のニーズに応える姿勢に期待しています。
選挙を通じて、政治家が地域振興に全力で取り組む姿勢が伝わってきます。具体的な政策提案を目にするたび、未来への希望が膨らみます。


## 出力例
{{
    "label": "市民の未来を支える具体的政策への期待",
    "description": "このクラスタには、地域の復興や被害者支援など、実際の社会課題に対して政治家が具体的かつ積極的に取り組む姿勢を支持する前向きな意見が集まっています。市民は、選挙や政策議論を通じて、現実の問題に即した支援策や復興計画が実現されることを期待し、明るい未来の構築に向けた政治の変革を応援しています。"
}}"""

In [98]:
from broadlistening.pipeline.steps.hierarchical_initial_labelling import initial_labelling

In [99]:
cluster_id_columns = [col for col in clusters_argument_df.columns if col.startswith("cluster-level-")]
initial_cluster_id_column = cluster_id_columns[-1]
sampling_num = 30
initial_labelling_prompt = INITIAL_PROMPT
model = "gpt-4o"
workers = 70

initial_label_df = initial_labelling(
    initial_labelling_prompt,
    clusters_argument_df,
    sampling_num,
    model,
    workers,
)

In [100]:
# 最下層のクラスタ。プロンプト修正後
initial_label_df.head()

,cluster_id,label,description
0,2_42,衆院選における落選候補者への注目と意見,このクラスタには、衆議院選挙における候補者の落選に関する様々な意見が集まっています。特に、落...
1,2_16,地域ごとの選挙結果と政党の勢力図,このクラスタには、各地域における選挙結果や政党の勢力に関する意見が集まっています。特に、立憲...
2,2_61,選挙に対する多様な関心と意見の集まり,このクラスタには、選挙に関連する様々な意見や関心が集まっています。選挙に対する肯定的な意見や...
3,2_20,低投票率と政治への関心の低さに対する懸念と提案,このクラスタには、選挙における低投票率や政治への関心の低さに対する懸念が集まっています。多く...
4,2_5,衆院選に対する多様な関心と期待,このクラスタには、衆議院選挙に関する多様な意見や反応が集まっています。選挙の結果や予測に対す...


In [101]:
initial_clusters_argument_df = clusters_argument_df.merge(
    initial_label_df,
    left_on=initial_cluster_id_column,
    right_on="cluster_id",
    how="left",
).rename(
    columns={
        "label": f"{initial_cluster_id_column.replace('-id', '')}-label",
        "description": f"{initial_cluster_id_column.replace('-id', '')}-description",
    }
)

In [102]:
initial_clusters_argument_df

,arg-id,argument,x,y,cluster-level-1-id,cluster-level-2-id,cluster_id,cluster-level-2-label,cluster-level-2-description
0,Acsv-1_0,当選確実の報道がある一方で、落選確実の状況もあるという意見がある。,9.395892,4.318309,1_15,2_42,2_42,衆院選における落選候補者への注目と意見,このクラスタには、衆議院選挙における候補者の落選に関する様々な意見が集まっています。特に、落...
1,Acsv-2_0,愛媛で自民党が1区と3区、立憲民主党が2区で当選したという意見があるが、政権交代がどう影響す...,6.425354,2.939700,1_9,2_16,2_16,地域ごとの選挙結果と政党の勢力図,このクラスタには、各地域における選挙結果や政党の勢力に関する意見が集まっています。特に、立憲...
2,Acsv-3_0,選挙の結果が気になって食事に行けないという声がある。,11.113695,5.199418,1_14,2_61,2_61,選挙に対する多様な関心と意見の集まり,このクラスタには、選挙に関連する様々な意見や関心が集まっています。選挙に対する肯定的な意見や...
3,Acsv-4_0,選挙に行かなかった人に対して消費税を20%にするという意見がある。,9.655325,7.321733,1_13,2_20,2_20,低投票率と政治への関心の低さに対する懸念と提案,このクラスタには、選挙における低投票率や政治への関心の低さに対する懸念が集まっています。多く...
4,Acsv-5_0,衆院選が始まり、選挙戦が面白くなってきたという意見がある。,9.691563,4.819920,1_14,2_5,2_5,衆院選に対する多様な関心と期待,このクラスタには、衆議院選挙に関する多様な意見や反応が集まっています。選挙の結果や予測に対す...
...,...,...,...,...,...,...,...,...,...
10129,Acsv-10085_0,自民党が議席を大幅に減らす可能性があるという意見がある。,3.502974,5.337620,1_8,2_55,2_55,自民党の議席減少に対する多様な反応と期待,このクラスタには、自民党の議席減少に関する様々な意見や反応が集まっています。自民党の議席減少...
10130,Acsv-10086_0,選挙速報を楽しんで見ているという意見がある。,12.083847,4.712981,1_12,2_40,2_40,選挙速報に対する多様な視点と感情,このクラスタには、選挙速報に対する様々な意見や感情が集まっています。選挙速報をエンターテイン...
10131,Acsv-10089_0,死刑囚が選挙権を持たないことに対する意見が述べられている。,10.587885,6.653868,1_13,2_0,2_0,選挙の重要性と投票行動への意識向上,このクラスタには、選挙に参加することの重要性を強調し、投票行動を促進する意見が集まっています...
10132,Acsv-10090_0,麻生太郎氏が衆院選で当選したとの情報がある。,6.751141,1.018640,1_11,2_24,2_24,選挙結果に対する多様な反応と疑念,このクラスタには、選挙結果や候補者に対する多様な意見や反応が集まっています。特に、麻生太郎氏...


In [103]:
initial_clusters_argument_df.to_csv(exp_output_dir / "hierarchical_initial_labels.csv", index=False)

## 統合ラベリング

In [104]:
from broadlistening.pipeline.steps.hierarchical_merge_labelling import _filter_id_columns, merge_labelling, melt_cluster_data, _build_parent_child_mapping, calculate_cluster_density

In [105]:
MERGE_PROMPT = """分割されすぎたクラスタを統合する必要があるので、統合後の名称を考えて出力して。

# 指示
* 統合前のクラスタの名称・説明および統合後のクラスタに属するデータ点のサンプルを与えるので、これらに基づいて統合後のクラスタの名称を出力してください
    * 統合後のクラスタ名において、統合前のクラスタ名をそのまま使うことは避けてください。
* 出力例に記載したJSONのフォーマットに従って出力してください

# サンプルの入出力
## 入力例（クラスタラベル:説明文）
- 地域の災害対応への批判: このクラスタは、地域における災害対応策の実施や体制に対する批判的な意見を集約したものです。住民からは、迅速かつ効果的な支援が行われていない点や、情報提供・連携の不足などに対する強い不満が表明されています。
- 災害対応への不満: このクラスタは、災害発生時の対応全般に対する不満を示す意見をまとめたものです。救援活動の遅れや支援策の実効性に疑問を持つ声が多く、より積極的で透明性のある対応を求める意見が特徴です。
- 地域復興の遅れ: このクラスタは、災害後の地域復興プロセスが予定通りに進んでいない点に対する懸念や不満を反映しています。再建計画や支援策の実施の遅延、そしてそれに伴う住民の生活再建への影響が強調されています。


## 出力例
{{
    "label": "地域再生と災害支援に対する期待と懸念",
    "description": "このクラスタは、特定の地域における再生や災害支援策に対し、具体的な取り組みが不足しているとの意見を集約しています。市民は、選挙を通じた政策議論の中で、地域復興や被災者支援を最優先すべきだとの期待と、現行の支援策に対する改善要求を強く表明しており、より効果的な政府の対応を求める声が反映されています。"
}}"""

In [106]:
config = {}
config["hierarchical_merge_labelling"] = {
    "prompt": MERGE_PROMPT,
    "model": "gpt-4o",
    "sampling_num": 50,
    "workers": 70,
}

In [107]:
clusters_df = initial_clusters_argument_df.copy()
cluster_id_columns: list[str] = _filter_id_columns(clusters_df.columns)
merge_result_df = merge_labelling(
    clusters_df=clusters_df,
    cluster_id_columns=sorted(cluster_id_columns, reverse=True),
    config=config,
)

100%|██████████| 1/1 [00:08<00:00,  8.82s/it]


In [108]:
melted_df = melt_cluster_data(merge_result_df)
parent_child_df = _build_parent_child_mapping(merge_result_df, cluster_id_columns)
melted_df = melted_df.merge(parent_child_df, on=["level", "id"], how="left")
melted_df.head(20)

,level,id,label,description,value,parent
0,1,1_15,選挙結果に対する市民の多様な反応と期待,このクラスタは、選挙結果に対する市民の多様な反応や期待を集約しています。市民は、候補者の当選...,691,0
1,1,1_9,地域選挙動向と市民の反応,このクラスタは、日本各地の選挙結果や政党支持の動向に対する市民の多様な反応を集約しています。...,708,0
2,1,1_14,選挙に対する多様な視点と社会的影響,このクラスタは、選挙に関連する多様な意見や関心を集約しています。市民の選挙に対する期待や不安...,699,0
3,1,1_13,選挙参加意識と制度改善への期待,このクラスタは、選挙に対する市民の多様な意見や体験を集約し、選挙参加の重要性と制度改善への期...,1299,0
4,1,1_4,選挙結果に対する多様な反応と期待,このクラスタは、最近の選挙結果に対する国民の多様な反応や期待を集約しています。特に、立憲民主...,681,0
5,1,1_10,選挙結果に対する多様な市民の反応と期待,このクラスタは、衆議院選挙結果に対する市民の多様な反応を集約しています。特定の政治家の当選や...,723,0
6,1,1_7,選挙における政党動向と市民の多様な反応,このクラスタは、選挙における政党の動向や市民の反応に関する多様な意見を集約しています。特に、...,603,0
7,1,1_12,選挙報道と視聴者の多様な視点と体験,このクラスタは、選挙報道に対する視聴者の多様な意見や体験を集約しています。視聴者は、選挙特番...,807,0
8,1,1_11,選挙結果に対する多様な反応と信頼性への懸念,このクラスタは、選挙結果に関する多様な意見や反応を集約しています。特に、選挙結果の信頼性や正...,692,0
9,1,1_3,日本の政治的再編と連立の行方に対する期待と懸念,このクラスタは、日本の主要政党間での連立の可能性やその影響についての意見を集約しています。特...,973,0


In [109]:
melted_df.to_csv(exp_output_dir / "プロンプト変更後のラベリング結果.csv", index=False)

In [110]:
# 自動評価

In [111]:
import json
import random
from concurrent.futures import ThreadPoolExecutor, as_completed
from tqdm import tqdm
random.seed(42)

from broadlistening.pipeline.services.llm import request_to_chat_openai

In [112]:
EVALUATION_PROMPT = """あなたはテキストクラスタリングの専門家です。クラスタリング結果のタイトルの適切性を評価してください。

# 指示
与えられた情報に基づいて、クラスタのタイトルがそのクラスタに含まれるテキストデータをどれだけ適切に表現しているかを評価してください。
評価結果は出力例に記載したjson形式で出力してください。

# 入力情報
1. 評価対象クラスタのタイトル
2. 評価対象クラスタに所属するテキストデータのサンプル
3. 他のクラスタからサンプリングしたテキストデータ

# 評価基準
以下の観点から、クラスタのタイトルの適切性を1〜5の5段階で評価してください：

1. inclusiveness_score: タイトルがクラスタ内のすべてのテキストの共通テーマを捉えているか（1: 全く捉えていない 〜 5: 完全に捉えている）
2. specificity_score: タイトルがこのクラスタを他のクラスタと明確に区別できるか（1: 全く区別できない 〜 5: 完全に区別できる）
3. concreteness_score: タイトルが抽象的すぎず、具体的な内容を示しているか（1: 非常に抽象的 〜 5: 非常に具体的）

# 評価例

## 入力例
評価対象クラスタのタイトル: 「災害対応の迅速化と情報共有の強化」

評価対象クラスタに所属するテキストデータのサンプル:
- 災害発生時の初動対応をもっと迅速にしてほしい。特に高齢者への支援が遅れがちだ。
- 避難所の情報がリアルタイムで更新されず、どこに行けばいいのか混乱した。
- 災害時の情報共有システムを強化し、住民が必要な情報にすぐアクセスできるようにすべき。
- 地域ごとの災害対策本部の連携が不十分で、支援物資の配布に無駄が生じていた。
- 災害発生から避難指示が出るまでのタイムラグを短縮する必要がある。

他のクラスタからサンプリングしたテキストデータ:
- 復興予算の使い道をもっと透明化してほしい。どこにいくら使われているのか不明確だ。
- 被災地域の経済復興のための長期的な計画が見えてこない。
- 防災教育を学校カリキュラムに積極的に取り入れるべきだ。
- ボランティアの受け入れ体制が整っておらず、せっかくの支援の手が活かされていない。

## 出力例
{{
  "inclusiveness_score": {{
      "score": 4,
      "reason": "タイトル「災害対応の迅速化と情報共有の強化」は、クラスタ内のテキストの主要なテーマである「初動対応の迅速化」「情報共有の問題」「対策本部の連携」などの要素を広くカバーしています。ただし、「高齢者への支援」という具体的な対象については明示されていないため、完全ではありません。"
  }},
  "specificity_score": {{
      "score": 4,
      "reason": "このタイトルは「対応の迅速化」と「情報共有」という2つの明確な焦点を持っており、他のクラスタ（予算の透明化、経済復興、防災教育など）とは明確に区別できます。ただし、災害対応の中でも「初動」に関する内容が多いため、もう少し特化した表現があるとより区別しやすくなります。"
  }},
  "concreteness_score": {{
      "score": 3,
      "reason": "「迅速化」と「情報共有の強化」という方向性は示されていますが、どのような迅速化（初動対応、避難指示など）なのか、どのような情報共有（避難所情報、支援物資など）なのかまでは具体化されていません。"
  }},
}}
"""

In [113]:
def sample_cluster_data(
    df: pd.DataFrame,
    cluster_column: str,
    cluster_id: str,
    text_column: str,
    sample_size: int
) -> list[str]:
    """クラスタからデータをサンプリングする

    Args:
        df: クラスタリング結果のDataFrame
        cluster_column: クラスタIDが格納されている列名
        cluster_id: サンプリング対象のクラスタID
        text_column: テキストデータが格納されている列名
        sample_size: サンプリングするデータ数

    Returns:
        サンプリングしたテキストデータのリスト
    """
    cluster_data = df[df[cluster_column] == cluster_id]
    sample_size = min(sample_size, len(cluster_data))
    
    if sample_size == 0:
        return []
    
    return cluster_data.sample(sample_size)[text_column].tolist()


def sample_other_clusters_data(
    df: pd.DataFrame,
    cluster_column: str,
    exclude_cluster_id: str,
    text_column: str,
    sample_size: int,
    sample_clusters: int | None = None
) -> list[str]:
    """他のクラスタからデータをサンプリングする

    Args:
        df: クラスタリング結果のDataFrame
        cluster_column: クラスタIDが格納されている列名
        exclude_cluster_id: 除外するクラスタID
        text_column: テキストデータが格納されている列名
        sample_size: 各クラスタからサンプリングするデータ数
        sample_clusters: サンプリングするクラスタ数（Noneの場合は全クラスタ）

    Returns:
        サンプリングしたテキストデータのリスト
    """
    other_clusters = df[df[cluster_column] != exclude_cluster_id][cluster_column].unique()
    
    if len(other_clusters) == 0:
        return []
    
    if sample_clusters is not None and sample_clusters < len(other_clusters):
        other_clusters = random.sample(list(other_clusters), sample_clusters)
    
    sampled_data = []
    for cluster_id in other_clusters:
        cluster_samples = sample_cluster_data(
            df, cluster_column, cluster_id, text_column, sample_size
        )
        sampled_data.extend(cluster_samples)
    
    return sampled_data


def evaluate_cluster_title(
    prompt: str,
    cluster_title: str,
    cluster_samples: list[str],
    other_clusters_samples: list[str],
    model: str = "gpt-4o"
) -> dict[str, dict]:
    """クラスタのタイトルを評価する

    Args:
        prompt: 評価用プロンプト
        cluster_title: 評価対象のクラスタタイトル
        cluster_samples: 評価対象クラスタのサンプルデータ
        other_clusters_samples: 他のクラスタのサンプルデータ
        model: 使用するLLMモデル名

    Returns:
        評価結果（JSON形式）
    """
    # 入力データの整形
    input_text = f"評価対象クラスタのタイトル: 「{cluster_title}」\n\n"
    
    input_text += "評価対象クラスタに所属するテキストデータのサンプル:\n"
    for sample in cluster_samples:
        input_text += f"- {sample}\n"
    
    input_text += "\n他のクラスタからサンプリングしたテキストデータ:\n"
    for sample in other_clusters_samples:
        input_text += f"- {sample}\n"
    
    # LLMに評価リクエスト
    messages = [
        {"role": "system", "content": prompt},
        {"role": "user", "content": input_text},
    ]
    
    try:
        response = request_to_chat_openai(messages=messages, model=model, is_json=True)
        evaluation = json.loads(response)
        return evaluation
    except Exception as e:
        print(f"評価中にエラーが発生しました: {e}")
        return {
            "error": str(e),
            "title_evaluation": {
                "inclusiveness_score": {"score": 0, "reason": "評価エラー"},
                "specificity_score": {"score": 0, "reason": "評価エラー"},
                "concreteness_score": {"score": 0, "reason": "評価エラー"},
                "overall_score": {"score": 0, "reason": "評価エラー"},
            }
        }

def evaluate(
    prompt: str,
    clusters_df: pd.DataFrame,
    cluster_column: str,
    text_column: str,
    sample_size: int,
    other_sample_size: int,
    other_sample_clusters: int | None = None,
    max_workers: int | None = None,  # 並列数を指定（Noneなら自動）
) -> list[dict[str, dict]]:
    cluster_ids = sorted(clusters_df[cluster_column].unique())
    evaluation_results = []

    def eval_one(cluster_id):
        cluster_title = clusters_df[clusters_df[cluster_column] == cluster_id]["cluster-level-2-label"].values[0]
        cluster_samples = sample_cluster_data(clusters_df, cluster_column, cluster_id, text_column, sample_size)
        other_clusters_samples = sample_other_clusters_data(
            clusters_df, cluster_column, cluster_id, text_column, other_sample_size, other_sample_clusters
        )
        return {
            "id": cluster_id,
            "label": cluster_title,
            **evaluate_cluster_title(prompt, cluster_title, cluster_samples, other_clusters_samples)
        }

    with ThreadPoolExecutor(max_workers=max_workers) as executor:
        futures = {executor.submit(eval_one, cluster_id): cluster_id for cluster_id in cluster_ids}
        for future in tqdm(as_completed(futures), total=len(futures)):
            result = future.result()
            evaluation_results.append(result)

    return evaluation_results


In [114]:
cluster_column = "cluster-level-1-id"
cluster_ids = sorted(clusters_df[cluster_column].unique())


In [115]:
eval_results = evaluate(
    prompt=EVALUATION_PROMPT,
    clusters_df=clusters_df,
    cluster_column="cluster-level-1-id",
    text_column="argument",
    sample_size=30,
    other_sample_size=4,
    other_sample_clusters=10
)

100%|██████████| 15/15 [00:11<00:00,  1.36it/s]


In [116]:
def parse_val(val: dict | str):
    if isinstance(val, dict):
        return val["score"]
    return val

In [117]:
eval_score_df = pd.DataFrame([
        {
            key: parse_val(val_dict)
            for key, val_dict in eval_result.items()} 
            for eval_result in eval_results
            
    ]
)

In [118]:
eval_score_df.sort_values(by="id")

,id,label,inclusiveness_score,specificity_score,concreteness_score
8,1_1,石破氏の政権運営と退陣に関する懸念と批判,5,4,4
9,1_10,選挙結果に対する多様な反応と疑問,4,3,3
12,1_11,選挙結果に関する不確実性と情報の信頼性,4,4,3
1,1_12,TBSのプロ野球と選挙同時中継に対する視聴者の多様な意見,3,4,3
13,1_13,低投票率と政治への関心の低さに対する懸念と提案,4,4,3
3,1_14,選挙に対する多様な関心と意見の集まり,5,3,4
6,1_15,衆院選における落選候補者への注目と意見,3,3,2
2,1_2,衆院選後の政権構成と過半数割れへの関心,5,4,4
5,1_3,自民党と公明党の過半数確保に対する不安と期待,5,4,4
0,1_4,民主党略称問題による投票混乱と公平性への懸念,3,4,3


In [119]:
eval_score_df[["inclusiveness_score", "specificity_score", "concreteness_score"]].mean(axis=0)

inclusiveness_score    4.200000
specificity_score      3.666667
concreteness_score     3.400000
dtype: float64

In [120]:
print("平均スコア", eval_score_df[["inclusiveness_score", "specificity_score", "concreteness_score"]].mean().mean())

平均スコア 3.755555555555556


In [121]:
eval_score_df.sort_values(by="id").to_csv(exp_output_dir / "ベースプロンプトの自動評価結果.csv", index=False)

In [122]:
eval_score_df

,id,label,inclusiveness_score,specificity_score,concreteness_score
0,1_4,民主党略称問題による投票混乱と公平性への懸念,3,4,3
1,1_12,TBSのプロ野球と選挙同時中継に対する視聴者の多様な意見,3,4,3
2,1_2,衆院選後の政権構成と過半数割れへの関心,5,4,4
3,1_14,選挙に対する多様な関心と意見の集まり,5,3,4
4,1_5,れいわ新選組の議席獲得に対する多様な意見と期待,5,4,4
5,1_3,自民党と公明党の過半数確保に対する不安と期待,5,4,4
6,1_15,衆院選における落選候補者への注目と意見,3,3,2
7,1_7,自民党政権に対する批判と政権交代への期待,5,4,4
8,1_1,石破氏の政権運営と退陣に関する懸念と批判,5,4,4
9,1_10,選挙結果に対する多様な反応と疑問,4,3,3
